## 1. Initialization

In [1]:
import sys
import os

os.chdir("../")
path1 = os.getcwd()

os.chdir("coppeliasim_utils")
path2 = os.getcwd()


sys.path.append(path1)
sys.path.append(path2)

from indy_utils import indydcp_client as client
from coppeliasim_utils import sim

from time import sleep
import time
import json
import threading
import numpy as np
import math
import keyboard
import threading

robot_ip = "192.168.0.15"  # Robot (Indy) IP
robot_name = "NRMK-Indy7"  # Robot name (Indy7)

# Create class object (real robot)
indy_actual = client.IndyDCPClient(robot_ip, robot_name)

# Crate class object (simulation robot)
indy_virtual = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)

# Actual robot (Created: True, Not created: False)
# Virtual robot (Created: True, Not created: False)
print("real robot: {0}\nsimulation robot: {1}".format(bool(indy_actual), bool(not indy_virtual)))

real robot: True
simulation robot: True


## 2. 로봇 관절 제어 함수

In [2]:
import math
from math import dist

# 각도를 라디안으로 변환하는 함수
def degrees_to_radians(degrees):
    return math.radians(degrees)

# 라디안을 각도로 변환하는 함수
def radians_to_degrees(radians):
    return math.degrees(radians)

# 로봇의 각 관절을 특정 각도로 이동시키는 함수
def move_joint_to_angle(jointHandle, targetAngle):
    sim.simxSetJointTargetPosition(indy_virtual, jointHandle, targetAngle, sim.simx_opmode_blocking)

# 로봇의 6개 관절을 각각 원하는 각도로 이동시키는 예시
def move_robot_to_angles(angles):
    # 각 관절을 원하는 각도로 이동시킴
    for i, angle in enumerate(angles):
        move_joint_to_angle(jointHandles[i], angle)

In [3]:
# 관절 핸들 선언
jointHandles = [0] * 6
jointNames = ['joint0', 'joint1', 'joint2', 'joint3', 'joint4', 'joint5']

# 관절 핸들 얻기
for i, jointName in enumerate(jointNames):
    res, jointHandles[i] = sim.simxGetObjectHandle(indy_virtual, jointName, sim.simx_opmode_blocking)
    print(res, jointHandles[i])
    
    
# 그리퍼(L, R)의 handle 가져오기
res, gripperLHandle = sim.simxGetObjectHandle(indy_virtual, 'joint_R', sim.simx_opmode_blocking)
print(res, gripperLHandle)

res, gripperRHandle = sim.simxGetObjectHandle(indy_virtual, 'joint_L', sim.simx_opmode_blocking)
print(res, gripperRHandle)

    
### For actual robot
indy_actual.connect()

# Zero position
j_pos_act = [0, 0, 0, 0, 0, 0]
indy_actual.joint_move_to(j_pos_act)  # Move 6th joint

indy_actual.disconnect()

#Zero position
j_pos_vir = [0, 0, 0, 0, 0, 0]
j_pos_rad = [degrees_to_radians(angle) for angle in j_pos_vir]
move_robot_to_angles(j_pos_rad)


# for i in range(6):
#     object_name = 'joint' + str(i)
#     #print(object_name)
#     result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
#     if result != sim.simx_return_ok:
#         raise Exception('could not get object handle for first joint')                
#     else:
#         handles.append(handle)
#         sim.simxSetJointTargetPosition(indy_virtual, handle, j_pos_vir[i]*math.pi/180, sim.simx_opmode_oneshot)

0 19
0 21
0 23
0 25
0 27
0 29
0 36
0 39
Connect: Server IP (192.168.0.15)


In [4]:
# 로봇의 각 관절을 특정 각도로 이동시키는 함수
def move_gripper_to_angle(jointHandle, targetAngle):
    sim.simxSetJointPosition(indy_virtual, jointHandle, targetAngle, sim.simx_opmode_blocking)

# 로봇의 6개 관절을 각각 원하는 각도로 이동시키는 예시
def move_gripper_to_angles(angles):
    # 각 관절을 원하는 각도로 이동시킴
    for i, angle in enumerate(angles):
        if i == 0:
            print(i, gripperLHandle, angle)
            move_gripper_to_angle(gripperLHandle, angle)
        elif i == 1:    
            print(i, gripperRHandle, angle)
            move_gripper_to_angle(gripperRHandle, angle)

## 3. Set zero position

## 4. Synchronization (w threaded)

In [8]:
def cps_program():
    indy_actual.connect()

    isCtrOn = True

    while isCtrOn:
        j_pos = indy_actual.get_joint_pos()
        time.sleep(0.002)

        for i in range(6):
            sim.simxSetJointTargetPosition(indy_virtual, jointHandles[i] , float(j_pos[i])*math.pi/180 ,sim.simx_opmode_oneshot)

        if keyboard.is_pressed("q"):
            isCtrOn = False
            print("Escape the program")

    indy_actual.disconnect()

t = threading.Thread(target=cps_program)
t.start()
print("Main Thread")


Main Thread
Connect: Server IP (192.168.0.15)


## 4-0. 근접센서 값 받기

In [9]:
result, toilet_handle_prx = sim.simxGetObjectHandle(indy_virtual, 'Toilet_Proximity_sensor', sim.simx_opmode_blocking)
print(result, toilet_handle_prx)
result, sink_handle_prx = sim.simxGetObjectHandle(indy_virtual, 'Sink_Proximity_sensor', sim.simx_opmode_blocking)
print(result, sink_handle_prx)

0 51
0 52


In [11]:
#변기 근접센서 위치
to_result, toilet_position = sim.simxGetObjectPosition(indy_virtual, toilet_handle_prx,-1, sim.simx_opmode_streaming)
print(to_result, toilet_position)

#싱크대 근접센서 위치
si_result, sink_position = sim.simxGetObjectPosition(indy_virtual, sink_handle_prx,-1, sim.simx_opmode_streaming)
print(si_result, sink_position)

state = 0# right

0 [-0.800000011920929, 0.30000001192092896, 0.15000000596046448]
0 [-0.17499999701976776, 0.5, 0.4000000059604645]


### 4-1. get joint data

In [12]:
j_pos_1 = [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18] #초기 위치
j_pos_2 = [-51.25, -43.29, -82.14, -0.88, -54.23, -51.30] #첫번째 타일 위치
j_pos_3 = [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18] #중간 위치1
j_pos_4 = [-125.09, -15.33, -73.99, -0.02, -89.35, -34.58] #중간 위치2
j_pos_5 = [-120.10, -45.61, -100.56, -0.25, -32.91, -28.71] #부착 위치1
j_pos_6 = [-125.09, -15.33, -73.99, -0.02, -89.35, -34.58] #중간 위치2
j_pos_7 = [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18] #중간 위치1
j_pos_8 = [-44.40, -47.90, -72.60, -0.90, -58.86, -44.76] #두번째 타일 위치
j_pos_9 = [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18] #중간 위치1
j_pos_9_2 = [30.61, -39.88, -60.81, -0.74, -89.21, -45.18] 
j_pos_10 = [90.61, -39.88, -60.81, -0.74, -89.21, -45.18] #싱크대 근접센서 작동 위치
j_pos_10_2 = [10.09, -15.33, -73.99, -0.02, -89.35, -34.58]
j_pos_10_3 = [-60.09, -15.33, -73.99, -0.02, -89.35, -34.58]
j_pos_11 = [-125.09, -15.33, -73.99, -0.02, -89.35, -34.58] #중간 위치2
j_pos_12 = [-125.07, -42.39, -109.41, -0.04, -26.88, -34.55] #부착 위치2
j_pos_13 = [-125.09, -15.33, -73.99, -0.02, -89.35, -34.58] #중간 위치2
j_pos_14 = [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18] #중간 위치1
j_pos_15 = [-38.76, -54.50, -58.95, -0.88, -65.37, -38.39] #세번째 타일 위치
j_pos_16 = [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18] #중간 위치1
j_pos_17 = [-125.09, -15.33, -73.99, -0.02, -89.35, -34.58] #중간 위치2
j_pos_18 = [-130.46, -40.05, -115.80, 0.14, -23.25, -39.74] #부착 위치3
j_pos_19 = [-125.09, -15.33, -73.99, -0.02, -89.35, -34.58] #중간 위치2
j_pos_20 = [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18] #초기 위치

j_pos = [j_pos_1, j_pos_2, j_pos_3, j_pos_4, j_pos_5, j_pos_6, j_pos_7, j_pos_8, j_pos_9, j_pos_9_2, j_pos_10, j_pos_10_2, j_pos_10_3, j_pos_11, j_pos_12, j_pos_13, j_pos_14, j_pos_15, j_pos_16, j_pos_17, j_pos_18, j_pos_19, j_pos_20]
# j_pos = [j_pos_8, j_pos_9, j_pos_9_2, j_pos_10, j_pos_10_2,j_pos_10_3, j_pos_11, j_pos_12, j_pos_13, j_pos_14, j_pos_15, j_pos_16, j_pos_17, j_pos_18, j_pos_19, j_pos_20]


# 그리퍼 이동 각도
gripper_hold = [0.012, 0.01]
gripper_release = [0.00, 0.00]


# j = 0
# for i in j_pos:
#     print(i)
#     j += 1
#     print(j)


In [13]:
def toilet(detected_THandle, j_pos_component):      
    #감지된 물체 위치
    detected_T_result, detected_T_position = sim.simxGetObjectPosition(indy_virtual, detected_THandle,-1, sim.simx_opmode_streaming)
    print(detected_T_position)
    
    #물체와 변기 사이 거리
    distance = math.dist(detected_T_position, toilet_position)
    
    #위치 조정
    NewX = detected_T_position[0] + 30 * math.cos(-60/180*3.14)
    NewY = detected_T_position[1] + 30 * math.sin(-60/180*3.14)
    print(NewX, NewY)
    newPosition = [NewX, NewY, detected_T_position[2]]

    #새로 구한 거리가 더 작을 때, 오른쪽으로 이동
    if(distance > dist(toilet_position, newPosition)):
        state = 0
    else:
        state = 1# left   

    if(state):#right
        #오른쪽으로 이동
        print("오른쪽으로 이동")
        j_pos_component[0] = j_pos_component[0] + 30  
        indy_actual.joint_move_to(j_pos_component)
        j_pos_a = indy_actual.get_joint_pos()
        print(j_pos_a)
        j_pos_a_rad = [degrees_to_radians(angle) for angle in j_pos_a]
        move_robot_to_angles(j_pos_a_rad) 
        time.sleep(3)       


    else:#left
        #왼쪽으로 이동
        print("왼쪽으로 이동")
        j_pos_component[0] = j_pos_component[0] - 30  
        indy_actual.joint_move_to(j_pos_component)
        j_pos_a = indy_actual.get_joint_pos()
        print(j_pos_a)
        j_pos_a_rad = [degrees_to_radians(angle) for angle in j_pos_a]
        move_robot_to_angles(j_pos_a_rad) 
        time.sleep(3)       


In [14]:
def sink(detected_SHandle, j_pos_component):    
    detected_S_result, detected_S_position = sim.simxGetObjectPosition(indy_virtual, detected_SHandle,-1, sim.simx_opmode_streaming)
    print(detected_S_position)

    distance = dist(detected_S_position, sink_position)

    NewX = detected_S_position[0] + 30 * math.cos(-60/180*3.14)
    NewY = detected_S_position[1] + 30 * math.sin(-60/180*3.14)
    print(NewX, NewY)

    newPosition = [NewX, NewY,detected_S_position[2]]
/
    if(distance > dist(sink_position, newPosition)):
        state = 1
    else:
        state = 0# left   

    if(state):
        #오른쪽으로 이동
        print("오른쪽으로 이동")
        j_pos_component[0] = j_pos_component[0] + 30  
        indy_actual.joint_move_to(j_pos_component)
        j_pos_a = indy_actual.get_joint_pos()
        print(j_pos_a)
        j_pos_a_rad = [degrees_to_radians(angle) for angle in j_pos_a]
        move_robot_to_angles(j_pos_a_rad) 
        time.sleep(3)       
    else:
        #왼쪽으로 이동
        print("왼쪽으로 이동")
        j_pos_component[0] = j_pos_component[0] - 30  
        indy_actual.joint_move_to(j_pos_component)
        j_pos_a = indy_actual.get_joint_pos()
        print(j_pos_a)
        j_pos_a_rad = [degrees_to_radians(angle) for angle in j_pos_a]
        move_robot_to_angles(j_pos_a_rad) 
        time.sleep(3)       

In [15]:
# #로봇 관절 값 라디안 변환
# indy_actual.joint_move_to(j_pos_16)
    
    
# #로봇 관절 값 받기
# j_pos_a = indy_actual.get_joint_pos()



# #로봇 관절 값 라디안 변환
# j_pos_a_rad = [degrees_to_radians(angle) for angle in j_pos_a]
# move_robot_to_angles(j_pos_a_rad) 
    

endtool_type = 0
indy_actual.set_endtool_do(endtool_type, 1)

In [16]:

for j_pos_component in j_pos:  
    #로봇 이동
    indy_actual.joint_move_to(j_pos_component)
    
    
    #로봇 관절 값 받기
    j_pos_a = indy_actual.get_joint_pos()
    print('가상', j_pos_a)
    print('실제', j_pos_component)   
    
    
    #로봇 관절 값 라디안 변환
    j_pos_a_rad = [degrees_to_radians(angle) for angle in j_pos_a]
    move_robot_to_angles(j_pos_a_rad)
    
    time.sleep(3)
    
    ## 그리퍼
    # endtool_type:
    # 0: NPN, 1: PNP, 2: Not use, 3: eModi
    endtool_type = 0

    if (j_pos_component == j_pos_2) or (j_pos_component == j_pos_8) or (j_pos_component == j_pos_15):
        indy_actual.set_endtool_do(endtool_type, 0)
        time.sleep(1)
        move_gripper_to_angles(gripper_hold)
    elif (j_pos_component == j_pos_5) or (j_pos_component == j_pos_12) or (j_pos_component == j_pos_18):
        indy_actual.set_endtool_do(endtool_type, 1)
        time.sleep(1)
        move_gripper_to_angles(gripper_release)
    ##
    
    
    #변기 근접 센서 값 받기
    toilet_result = sim.simxReadProximitySensor(indy_virtual, toilet_handle_prx, sim.simx_opmode_blocking)
    detected_THandle = toilet_result[3]
    
#     if(j_pos_component == j_pos_10):
#         print("변기 근접센서 켁켓...")
#         toilet(detected_THandle, j_pos_component)
        
    #싱크대 근접센서 값
    sink_result = sim.simxReadProximitySensor(indy_virtual, sink_handle_prx, sim.simx_opmode_oneshot)
    print(sink_result[1], sink_result[3])
    detected_SHandle = sink_result[3]
    
    if(j_pos_component == j_pos_10):
        print("싱크대 근접센서 켁켓...")
        print("j_pos_component", j_pos_component)
        sink(detected_SHandle, j_pos_10)
        time.sleep(3)


가상 [-0.000408582450929779, -0.0, 4.539805010330878e-05, -5.438776299505309e-05, 5.438776299505309e-05, -5.438776299505309e-05]
실제 [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18]
False 0
가상 [-44.42952810240478, -39.89122122772732, -50.81549021823682, -0.7668674582302486, -89.19255927058137, -45.18372186340025]
실제 [-51.25, -43.29, -82.14, -0.88, -54.23, -51.3]
0 36 0.012
1 39 0.01
False 50
가상 [-51.25035814017642, -43.282727958745106, -82.14014478951988, -0.8791238010520381, -54.23047358447139, -51.30006091429793]
실제 [-44.42, -39.88, -50.81, -0.77, -89.19, -45.18]
False 0
가상 [-44.41690744447607, -39.882686394307896, -50.80736396726833, -0.7692061320390358, -89.19043814782457, -45.17888135249369]
실제 [-125.09, -15.33, -73.99, -0.02, -89.35, -34.58]
False 0
가상 [-125.10240791258572, -15.329559578384275, -73.98679118511593, -0.018274288366337838, -89.34789072169524, -34.57604134437109]
실제 [-120.1, -45.61, -100.56, -0.25, -32.91, -28.71]
0 36 0.0
1 39 0.0
False 0
가상 [-120.09745448089635, -45.60

## Communication reset

In [ ]:
# Reset TCP-IP communication (real robot)
# For real robot
indy_actual.connect()
indy_actual.disconnect()

# For simulation robot
sim.simxFinish(indy_virtual)

In [ ]:
# endtool_type:
# 0: NPN, 1: PNP, 2: Not use, 3: eModi

endtool_type = 0
indy_actual.set_endtool_do(endtool_type, 1)  # val: 0(off), 1(on)
sleep(1)
indy_actual.get_endtool_do(endtool_type)